<a href="https://colab.research.google.com/github/ErikSeguinte/movie_data/blob/master/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import cufflinks as cf
import numpy as np
from plotly import graph_objs as go
import altair as alt

In [ ]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [ ]:
cf.set_config_file(offline=True)

* I previously pulled CSV files from Kaggle, but the files were too big to host on github.
* I imported the files I wanted into pandas, and then exported them back out as compressed pickles.
* I was able to compress a 700MB csv to a 3 MB Pickle

In [ ]:
try: 
    movies = pd.read_pickle('data/movies.pkl.xz')
    ratings = pd.read_pickle('data/ratings2.pkl.xz')
    cpi = pd.read_csv('data/cpi.csv')
    
except:
    # Download pickles from github
    !wget https://github.com/ErikSeguinte/movie_data/raw/master/data/ratings2.pkl.xz
    !wget https://github.com/ErikSeguinte/movie_data/raw/master/data/movies.pkl.xz
    !wget 'https://datahub.io/core/cpi/r/cpi.csv'
    cpi = pd.read_csv('data/cpi.csv')
    movies = pd.read_pickle('movies.pkl.xz')
    ratings = pd.read_pickle('ratings2.pkl.xz')

## Clean Movie DF

In [ ]:
movies.dtypes

* Movies Dataframe has malformed data. `id` Should be numeric.
* After inspection, it looks like there are rows that are missing a comma somewhere, making columns not line up, and adding the wrong data to columns. Let's clean those up.
* All malformed rows have strings for IDs instead of numeric, so we will coerce them into numeric columns, and strings will be returned as `NaN`, which we'll then drop.

* `budget` and `revanue` should also be numeric, but Nans won't be dropped






In [ ]:
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')
movies = movies[movies['id'].notnull()]
movies = movies.set_index('id')

In [ ]:
def to_numeric(df, labels):
    
    for label in labels:
        df[label] = pd.to_numeric(movies[label], errors='coerce').copy()
    return df

In [ ]:
movies = to_numeric(movies, ['budget', 'revenue', 'vote_average'])

In [ ]:
movies['release_date'] =pd.to_datetime(movies['release_date'], infer_datetime_format= True)

In [ ]:
clean_movies = movies[['title', 'release_date','budget', 'revenue', 'runtime', 'vote_average', 'vote_count']]

## Process User Reviews
* User reviews come in a collection of individual reviews where a review gives a movie a score of 1 to 5.
* We will take the mean ratings for each movie

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
mean_rating = pd.DataFrame(scaler.fit_transform(ratings.groupby('movieId')[['rating']].mean()), columns = ["rating"])
#mean_rating = pd.DataFrame(ratings.groupby('movieId')[['rating']].mean(), columns = ["rating"])

In [ ]:
# Aggregate mean ratings and number of votes per movie
movie_ratings =pd.DataFrame(ratings.groupby('movieId')[['rating']].agg(['mean', 'count']))['rating']
movie_ratings = movie_ratings.rename({'mean': 'rating', 'count': 'num_votes'}, axis = 1)


In [ ]:
ratings = None

* Lets drop any movies with less than 10 votes. Those are more easily swayed by outliers and aren't reliable.

In [ ]:
movie_ratings = movie_ratings[~(movie_ratings['num_votes'] < 100)]

* And now we merge the averaged ratings back with the movie database.
* Note that not all movies are present in the user votings.

In [ ]:
movie_ratings = clean_movies.merge(movie_ratings, left_index = True, right_index=True)

In [ ]:
movie_ratings[['title', 'rating']].nlargest(10, 'rating')

* The movie Database also provides a rating and suffer from a similar problem of some movies having a tiny sample size.

In [ ]:
movie_ratings[['title', 'vote_average', 'vote_count']].sort_values(by='vote_average', ascending = False).nlargest(10, 'vote_average')

In [ ]:
movie_ratings[['title', 'revenue']].nlargest(10, 'revenue')

## Inflation
* Inflation means that a 1940 dollar is worth more than a 2020 dollar. Let's adjust Revenue for that.
* The Consumer price index can be used to convert to standarized dollars.
* Here, we'll be using 2014 dollars.
* Years later than 2014 will not be adjusted.
$$ \textrm{adjusted dollars} = \frac{\textrm{New CPI}}{\textrm{Base CPI}}$$
* where x is the current cpi and y is the cpi of that year 

In [ ]:
!wget 'https://datahub.io/core/cpi/r/cpi.csv'

In [ ]:
cpi = cpi[cpi['Country Name'] == 'United States'][['Year', 'CPI']]

In [ ]:
cpi = cpi.set_index(cpi['Year'])

In [ ]:
def adjust_dollars(value, year):
    year = int(year)
    try:
        current = cpi.loc[2014,'CPI']
        base = cpi.loc[year,'CPI']
        adjusted_value = value * (current/base)
        return adjusted_value
    except: 
        return value

In [ ]:
clean_movies['year']= clean_movies['release_date'].dt.year

In [ ]:
df = clean_movies[clean_movies['revenue'].notnull() & clean_movies['year'].notnull()]
df

In [ ]:
adjusted = pd.DataFrame([adjust_dollars(x,y) for x,y in zip(df['revenue'], df['year'])], index = df.index, columns = ['adjusted_revenue'])

In [ ]:
clean_movies.merge(adjusted, left_index=True, right_index=True).nlargest(10,'adjusted_revenue')

In [ ]:
movie_ratings.corr()

In [ ]:
# Adding a year and decade to examine trends over time
movie_ratings['year'] = movie_ratings['release_date'].dt.year

In [ ]:
movie_ratings['decade'] = [x - (x%10) for x in movie_ratings['year']]

In [ ]:
#enable_plotly_in_cell()
movie_ratings.groupby('year')['vote_average'].mean().iplot(kind='bar')

In [ ]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(vote_average)'),
    alt.X('year')
)

In [ ]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(rating)'),
    alt.X('year')
)

In [ ]:
alt.Chart(movie_ratings).mark_bar().encode(
    alt.Y('mean(vote_average)'),
    alt.X('decade')
)

In [ ]:
movie_ratings

In [ ]:
# enable_plotly_in_cell()
movie_ratings.groupby('decade')['rating'].mean().iplot(kind='bar')

* I'd like to compare the votes from TMB to the user ratings, but they are on different scales. We'll use standard scaler to normalize them so we can more easily compare.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
to_scale = movie_ratings[movie_ratings['vote_average'].notnull() & movie_ratings['year'].notnull() & movie_ratings['rating'].notnull()][['vote_average', 'rating']]
scaled = scaler.fit_transform(to_scale)
movie_ratings = movie_ratings.merge(
    pd.DataFrame(
        scaled,
        index = to_scale.index,
        columns = ['scaled_tmdb_vote', 'scaled_user_rating']
    ),
    left_index = True,
    right_index = True,
)

In [ ]:
movie_ratings

In [ ]:
traces = [
    go.Bar(name='TMDB rating',
        x = movie_ratings.groupby('year')['scaled_tmdb_vote'].mean().index,
        y = movie_ratings.groupby('year')['scaled_tmdb_vote'].mean()
    ),
        go.Bar(name='user rating',
        x = movie_ratings.groupby('year')['scaled_user_rating'].mean().index,
        y = movie_ratings.groupby('year')['scaled_user_rating'].mean()
    )
]

go.Figure(data = traces,
    layout_xaxis_tick0 = 1890
)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(1)

pca_df = pd.DataFrame(pca.fit_transform(scaled), index=to_scale.index, columns = ['PCA'])

movie_ratings = movie_ratings.merge(pca_df, left_index = True, right_index = True)
movie_ratings.head(2)

In [ ]:
movie_ratings[['title', 'PCA']].nlargest(10, 'PCA')

In [ ]:
# enable_plotly_in_cell()
trace = go.Box(
    x = movie_ratings[movie_ratings['decade'].notnull()]['decade'],
    y = movie_ratings[movie_ratings['decade'].notnull()]['rating'],
    
)
go.Figure(
    trace,
    layout_xaxis_title = "Decade",
    layout_yaxis_title = "Movie Rating",
    layout_title = "Movie Ratings by decade"
)


In [ ]:
movie_ratings['q_budget'] = pd.qcut(movie_ratings['budget'], labels = ['vlow', 'low', 'med', 'high', 'vhigh'], q = 5)

In [ ]:
budget_ratings = movie_ratings[['title', 'budget', 'revenue', 'rating']].dropna()

In [ ]:
budget_ratings.corr()

In [ ]:
trace = go.Scatter(
    y = budget_ratings['rating'],
    x = budget_ratings['revenue'],
    mode = 'markers'
)

go.Figure(
    trace,
    layout_xaxis_title = "Budget",
    layout_yaxis_title = "Movie Rating",
    layout_title = "Movie Ratings by budget",
    
)

In [ ]:
movie_ratings.shape

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA

In [ ]:
all_ratings = movie_ratings[['vote_average','rating']].dropna()
all_ratings

In [ ]:
all_ratings.isnull().sum()

In [ ]:
scaler = StandardScaler()
x =scaler.fit_transform(all_ratings)

In [ ]:
pca = PCA(1)

In [ ]:
x = pca.fit_transform(x)

In [ ]:
scaled_ratings = pd.DataFrame(x, index = all_ratings.index, columns=['scaled_rating'])

In [ ]:
scaled_ratings

In [ ]:
movie_ratings = movie_ratings.merge(scaled_ratings, left_index=True, right_index=True)

In [ ]:
movie_ratings.corr()

In [ ]:
enable_plotly_in_cell()
movie_ratings.groupby('year')['scaled_rating'].mean().iplot(kind='bar')

In [ ]:
movie_ratings.groupby('year')['scaled_rating'].mean().iplot(kind='bar', title = "Scaled Rating by Year", xTitle="year", yTitle="Scaled Rating")

In [ ]:
movie_ratings.groupby('decade')['scaled_rating'].mean().iplot(kind='bar')

In [ ]:
budget_ratings = movie_ratings[['budget', 'rating']].dropna()

In [ ]:
budget_ratings['q_budget'] = pd.qcut(budget_ratings['budget'], q = 5, labels = ['vlow', 'low', 'med', 'high', 'blockbuster'])


In [ ]:
enable_plotly_in_cell()
budget_ratings.groupby('q_budget')['rating'].mean().iplot(kind='bar')

In [ ]:
movie_ratings.nlargest(25, 'rating')

In [ ]:
enable_plotly_in_cell()
trace = go.Scatter(
    x = movie_ratings["budget"],
    y = movie_ratings['revenue'],
    mode = "markers"
)

go.Figure(trace)